In [1]:
!uv add dspy-ai
!uv add pandas
!uv add numpy
!uv add openai
!uv add tqdm
!uv add scikit-learn
!uv add datasets
!uv add matplotlib


Resolved 140 packages in 480ms                                       
⠙ Preparing packages... (0/3)                                                   
⠙ Preparing packages... (0/3)----     0 B/10.18 MiB                     
json-repair ------------------------------     0 B/28.62 KiB
⠙ Preparing packages... (0/3)----     0 B/10.18 MiB                     
json-repair ------------------------------ 14.88 KiB/28.62 KiB
⠙ Preparing packages... (0/3)----     0 B/10.18 MiB                     
json-repair ------------------------------ 28.62 KiB/28.62 KiB
⠙ Preparing packages... (0/3)----     0 B/10.18 MiB                     
dspy-ai    ------------------------------     0 B/1.07 KiB
json-repair ------------------------------ 28.62 KiB/28.62 KiB
⠙ Preparing packages... (0/3)----     0 B/10.18 MiB                     
dspy-ai    ------------------------------ 1.07 KiB/1.07 KiB
json-repair ------------------------------ 28.62 KiB/28.62 KiB
⠙ Preparing packages... (0/3)----     0 B/10.18 MiB 

In [2]:
import pandas as pd
import json
import numpy as np
from openai import OpenAI
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import dspy
import random

# 1. 데이터 로드 및 전처리
print("데이터셋 로딩 중...")
ds = load_dataset("birdsql/bird_sql_dev_20251106")
df = ds['dev_20251106'].to_pandas()
hard_df = df[df['difficulty'] == 'challenging'].copy()
print(f"로드 완료. 'challenging' 난이도 데이터 {len(hard_df)}개.")

# 2. 훈련/검증/테스트 데이터 분리
print("데이터를 훈련셋, 검증셋, 테스트셋으로 분리 중...")
train_df, test_df = train_test_split(hard_df, test_size=0.2, random_state=42)
train_df, dev_df = train_test_split(train_df, test_size=0.25, random_state=42) # train_df의 25%를 dev로 사용 (전체의 20%)

print(f"훈련셋: {len(train_df)}개, 검증셋: {len(dev_df)}개, 테스트셋: {len(test_df)}개")

# DSPy Example 객체로 변환
train_set = [dspy.Example(question=row['question'], evidence=str(row['evidence']), sql=row['SQL']).with_inputs('question') for _, row in train_df.iterrows()]
dev_set = [dspy.Example(question=row['question'], evidence=str(row['evidence']), sql=row['SQL']).with_inputs('question') for _, row in dev_df.iterrows()]
test_set = [dspy.Example(question=row['question'], evidence=str(row['evidence']), sql=row['SQL']).with_inputs('question') for _, row in test_df.iterrows()]

print(f"DSPy용 데이터셋 생성 완료.")
print(f"Train_set 예시: {train_set[0]}")

# 3. OpenAI 클라이언트 설정
print("OpenAI 클라이언트 설정 중...")
MODEL_NAME="Qwen/Qwen3-Next-80B-A3B-Instruct"
MODEL_BASE_URI="http://211.47.56.81:7972/v1"
MODEL_API_KEY="token-abc123"
client = OpenAI(api_key=MODEL_API_KEY, base_url=MODEL_BASE_URI)
print("설정 완료.")


데이터셋 로딩 중...


README.md: 0.00B [00:00, ?B/s]

dev_20251106-00000-of-00001.json: 0.00B [00:00, ?B/s]

Generating dev_20251106 split: 0 examples [00:00, ? examples/s]

로드 완료. 'challenging' 난이도 데이터 231개.
데이터를 훈련셋, 검증셋, 테스트셋으로 분리 중...
훈련셋: 138개, 검증셋: 46개, 테스트셋: 47개
DSPy용 데이터셋 생성 완료.
Train_set 예시: Example({'question': "For molecule TR000, what are the bond types ranked by frequency, how many times does each bond type occur, and how many unique element pairs are connected by each bond type? Also indicate whether each bond type appears multiple times or just once, and show the molecule's label.", 'evidence': 'TR000 is the molecule id; bond frequency refers to whether bond_count is greater than 1', 'sql': "WITH bond_counts AS (\n    SELECT \n        b.molecule_id,\n        b.bond_type,\n        COUNT(*) AS bond_count,\n        ROW_NUMBER() OVER (PARTITION BY b.molecule_id ORDER BY COUNT(*) DESC) AS rank\n    FROM bond b\n    WHERE b.molecule_id = 'TR000'\n    GROUP BY b.molecule_id, b.bond_type\n),\nconnected_atoms AS (\n    SELECT \n        c.bond_id,\n        c.atom_id,\n        c.atom_id2,\n        a1.element AS element1,\n        a2.element AS element2

In [ ]:
# 4. DSPy LLM 설정
print("DSPy LLM 설정 중...")
# vLLM으로 서빙되는 로컬 모델을 litellm이 OpenAI 호환 엔드포인트로 인식하도록
# 모델 이름에 'openai/' 접두사를 추가합니다.
# 이렇게 하면 litellm이 provider를 'openai'로 올바르게 설정하고,
# 실제 API 요청 시에는 접두사를 제외한 원래 모델 이름(`Qwen/Qwen3-Next-80B-A3B-Instruct`)을 전송합니다.
llm = dspy.LM(
    model=f"openai/{MODEL_NAME}",
    api_base=MODEL_BASE_URI,
    api_key=MODEL_API_KEY,
    max_tokens=4000,
    temperature=0.0
)
dspy.configure(lm=llm)
print("설정 완료.")


DSPy LLM 설정 중...
설정 완료.


In [ ]:
# 5. DSPy 시그니처 및 모듈 정의

class Text2SQL(dspy.Signature):
    """주어진 자연어 질문을 SQL 쿼리로 변환합니다. 
    질문의 의도를 정확히 파악하여, 올바른 결과를 반환하는 SQL 쿼리를 작성해야 합니다."""

    question = dspy.InputField(desc="사용자의 자연어 질문")
    sql_query = dspy.OutputField(desc="생성된 SQL 쿼리")

class SQLAgent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.text_to_sql = dspy.Predict(Text2SQL)

    def forward(self, question):
        """질문을 받아 SQL 쿼리를 생성합니다."""
        prediction = self.text_to_sql(question=question)
        return dspy.Prediction(sql_query=prediction.sql_query)

print("DSPy Signature와 Module 정의 완료.")


DSPy Signature와 Module 정의 완료.


In [9]:
# 6. LLM 기반 평가 메트릭 정의

evaluation_prompt_template = """당신은 SQL 쿼리 평가 전문가입니다. 당신의 임무는 생성된 SQL 쿼리의 실행 결과가 정답(Evidence)과 일치하는지 평가하고, 정량적인 보상 점수와 상세한 피드백을 제공하는 것입니다.

**평가 기준:**
- **+3점 (완벽히 정답):** 실행 결과가 정답(Evidence)과  SQL(정답) 모든 면에서(데이터, 순서, 형식) 완벽하게 일치합니다.
- **+1점 (정답):** 실행 결과가 정답(Evidence)과 모든 면에서 일치하지만, SQL(정답)과 출력 순서가 다르거나 사소한 형식 차이가 있습니다.
- **+0.7점 (의미상 정답):** 데이터는 정확하지만, Evidence와 출력 순서가 다르거나 사소한 형식 차이가 있습니다.
- **-0.5점 (실행은 되나 오답):** SQL 쿼리가 문법적으로는 맞아 실행되지만, 질문의 의도를 잘못 파악하여 틀린 결과를 반환합니다.
- **-1.0점 (실행 불가 또는 심각한 오류):** SQL 문법 오류가 있어 실행되지 않거나, 질문의 핵심 요구사항을 완전히 놓쳤습니다.

**평가 대상 정보:**
1. **User Query (사용자 질문):** {query}
2. **Evidence (정답 데이터):** {evidence}
3. **SQL (정답):** {sql}
4. **LLM Result (생성된 SQL 쿼리):** {llm_result}

**출력 형식:**
평가 결과를 다음 형식에 맞춰 **오직 JSON 객체 하나만** 출력해주세요. 당신의 의견이나 다른 설명은 절대 포함하지 마세요.
```json
{{
  "reward_score": "평가 기준에 따른 숫자 점수",
  "correctness": "'Correct', 'Partially Correct', 'Incorrect' 중 하나",
  "feedback": "점수를 매긴 이유와, 쿼리를 개선하기 위한 구체적이고 건설적인 피드백을 작성해주세요."
}}
```
"""

def llm_based_metric(gold, pred, trace=None):
    """LLM을 사용하여 생성된 SQL을 평가하고 보상 점수를 반환하는 메트릭 함수"""
    try:
        # gold: dspy.Example (question, evidence, sql)
        # pred: dspy.Prediction (sql_query)
        user_query = gold.question
        evidence = gold.evidence
        true_sql = gold.sql
        llm_result = pred.sql_query

        prompt = evaluation_prompt_template.format(
            query=user_query,
            evidence=evidence,
            sql=true_sql,
            llm_result=llm_result
        )
        
        response = client.chat.completions.create(
            model=MODEL_NAME, 
            messages=[{"role": "system", "content": prompt}],
            temperature=0.0
        )
        
        eval_result_str = response.choices[0].message.content
        if "```json" in eval_result_str:
            clean_str = eval_result_str.split("```json\\n")[1].split("\\n```")[0]
        else:
            clean_str = eval_result_str
            
        eval_result = json.loads(clean_str)
        reward_score = float(eval_result.get('reward_score', -1.0))
        
        return reward_score
    except Exception as e:
        print(f"메트릭 평가 중 오류 발생: {e}")
        return -1.0

# 메트릭 함수 테스트
print("메트릭 함수 테스트...")
# dev_set에서 예제 하나를 가져와 테스트
example = dev_set[0]
prediction = dspy.Prediction(sql_query="SELECT COUNT(*) FROM table")
score = llm_based_metric(example, prediction)
print(f"테스트 평가 점수: {score}")
print("메트릭 정의 완료.")


메트릭 함수 테스트...
메트릭 평가 중 오류 발생: list index out of range
테스트 평가 점수: -1.0
메트릭 정의 완료.


In [10]:
# 7. DSPy 텔레프롬프터(Teleprompter)를 사용한 최적화

# BootstrapFewShot: 훈련셋을 사용하여 높은 점수를 받는 예제를 찾아내고,
# 이를 Few-shot 예제로 사용하여 에이전트의 프롬프트를 강화(최적화)합니다.
from dspy.teleprompt import BootstrapFewShot

# 성공의 기준을 점수로 정의 (1.0 이상이면 성공)
def metric_for_bootstrap(gold, pred, trace=None):
    score = llm_based_metric(gold, pred)
    if score is None: return False
    return score >= 1.0

# 텔레프롬프터 설정
teleprompter = BootstrapFewShot(
    metric=metric_for_bootstrap,
    max_bootstrapped_demos=4, # 몇 개의 성공 사례를 few-shot 예제로 만들 것인가
    max_labeled_demos=8 # 레이블된(성공/실패) 예제를 몇 개까지 참고할 것인가
)

# 최적화 실행
# train_set으로 few-shot 예제를 생성합니다.
print("최적화 시작...")
optimized_agent = teleprompter.compile(SQLAgent(), trainset=train_set)
print("최적화 완료.")


최적화 시작...


  0%|          | 0/138 [00:00<?, ?it/s]2025/11/23 13:06:24 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'question': "For molecule TR000, what are the bond types ranked by frequency, how many times does each bond type occur, and how many unique element pairs are connected by each bond type? Also indicate whether each bond type appears multiple times or just once, and show the molecule's label.", 'evidence': 'TR000 is the molecule id; bond frequency refers to whether bond_count is greater than 1', 'sql': "WITH bond_counts AS (\n    SELECT \n        b.molecule_id,\n        b.bond_type,\n        COUNT(*) AS bond_count,\n        ROW_NUMBER() OVER (PARTITION BY b.molecule_id ORDER BY COUNT(*) DESC) AS rank\n    FROM bond b\n    WHERE b.molecule_id = 'TR000'\n    GROUP BY b.molecule_id, b.bond_type\n),\nconnected_atoms AS (\n    SELECT \n        c.bond_id,\n        c.atom_id,\n        c.atom_id2,\n        a1.element AS element1,\n        a2.element AS element

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=Qwen/Qwen3-Next-80B-A3B-Instruct
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [ ]:
# 8. 최적화된 에이전트 성능 평가
from dspy.evaluate import Evaluate

# 평가를 위한 Evaluator 객체 생성
# num_threads를 사용하여 병렬로 빠르게 평가
evaluator = Evaluate(devset=test_set, num_threads=10, display_progress=True)

# 1) 최적화 이전 기본 에이전트 평가
print("최적화 이전 에이전트 평가 중...")
unoptimized_agent = SQLAgent()
unoptimized_score = evaluator(unoptimized_agent, metric=llm_based_metric)
print(f"최적화 이전 평균 점수: {unoptimized_score}")


# 2) 최적화된 에이전트 평가
print("\\n최적화 이후 에이전트 평가 중...")
optimized_score = evaluator(optimized_agent, metric=llm_based_metric)
print(f"최적화 이후 평균 점수: {optimized_score}")

print("\\n\\n========== 평가 요약 ==========")
print(f"최적화 이전 (Zero-shot): {unoptimized_score:.3f}")
print(f"최적화 이후 (Few-shot): {optimized_score:.3f}")
print("==============================")

